### Introduction to VivaBench

Thanks for checking out VivaBench! This notebook will introduce the core concepts of VivaBench, a multi-turn benchmark for LLM agents to assess their ability to form predictions and diagnose clinical conditions from incomplete information.

In [2]:
import os
from langchain.chat_models import init_chat_model
from vivabench import ClinicalCase


#### Structured Clinical Case
In VivaBench, clinical vignettes are structured as a `ClinicalCase`, which is the data format our evaluation framework use to benchmark LLMs.

Each Clinical Case is structured into five sections - History, Physical Examination, Investigations, Imaging, and Diagnosis / Differentials.

We further divide history into symptoms and other ancillary history items such as past medical history, social history etc.

In [3]:
# Load sample case
with open('example.json', 'r') as f:
    clinical_case_json = f.read()
    
clinical_case = ClinicalCase.model_validate_json(clinical_case_json)

# For each case, full information is provided for clinician review
print(clinical_case.full_information)

67 year old male presenting with sudden severe dizziness, unsteadiness, and double vision. 
==== Symptoms
### Neurological
## Dizziness
- Present: True
- Onset: sudden, two hours prior to arrival
- Duration: persistent since onset
- Progression: constant
- Severity: 8/10
- Character: severe spinning sensation (vertigo)
- Aggravating factors: standing, moving his head
- Associated symptoms: Nausea, Vomiting, Unsteady gait, Diplopia
- Context: occurred while watching television
- History: sudden, severe spinning sensation described as vertigo, constant and rated 8/10 in severity, associated with nausea and vomiting, worsened by standing or moving his head
## Unsteady gait
- Present: True
- Onset: sudden, two hours prior to arrival
- Duration: persistent since onset
- Associated symptoms: Dizziness, Diplopia
- Context: occurred while watching television
## Paresthesia
- Present: True
- Onset: sudden, two hours prior to arrival
- Duration: persistent since onset
- Location: left side of fa

#### Information Parsing and Retrieval
A key feature for VivaBench is its ability to perform multi-turn testing in a deterministic / quasi-determinstic manner, because of how we structure clinical information.

Each time an agent asks for information, it is first matched with the most suitable items via SNOMED mapping, before parsing back into free-text responses. This prevents leakage while preserving a realistic simulation environment

We provide determinstic and pseudo-deterministic (LLM) versions of mapper / parser. LLM versions have more flexibility but are not strictly deterministic (although we have tested it very thoroughly!)

In [19]:
# Here, we can see that each item associated with the patient's presentation are nicely structured into keys-value pairs
clinical_case.dict()

{'history:symptoms:dizziness': 'Dizziness',
 'history:symptoms:unsteady_gait': 'Unsteady gait',
 'history:symptoms:diplopia': 'Diplopia',
 'history:symptoms:nausea': 'Nausea',
 'history:symptoms:vomiting': 'Vomiting',
 'history:symptoms:paresthesia': 'Paresthesia',
 'history:symptoms:paresthesia_of_upper_limb': 'Paresthesia of upper limb',
 'history:symptoms:weakness_of_right_upper_limb': 'Weakness of right upper limb',
 'history:social_history:smoking_pack_years': '30.0',
 'history:social_history:smoking_quit': '5 years ago',
 'history:social_history:alcohol_use': 'Occasional alcohol consumption',
 'history:social_history:living_situation': 'Lives with spouse',
 'history:social_history:independent_in_activities_of_daily_living': '',
 'history:past_medical_history:hypertension': 'hypertension',
 'history:past_medical_history:type_2_diabetes_mellitus': 'type 2 diabetes mellitus',
 'history:past_medical_history:hyperlipidemia': 'hyperlipidemia',
 'history:past_medical_history:stroke': 's

In [ ]:
from vivabench.mapper import DeterminsticMapper, LLMMapper
from vivabench.parser import DeterminsticParser, LLMParser

In [23]:
model = init_chat_model("openai:gpt-4.1", temperature=0, api_key=os.environ['OPENAI_API'])

det_mapper = DeterminsticMapper(clinical_case, snomed_embeddings_path="../medical/snomed_embeddings")
llm_mapper = LLMMapper(clinical_case, model=model)

In [38]:
# For each request, the mapper returns set of matched / unmatched keys, and the relevant query
query = "Do you have any dizziness, nausea, or vomiting? How about any chest pain?"
requests = llm_mapper.map_history_requests(query)
requests

{'matched': [{'query': 'Do you have any dizziness, nausea, or vomiting?',
   'key': 'symptoms:dizziness'},
  {'query': 'Do you have any dizziness, nausea, or vomiting?',
   'key': 'symptoms:nausea'},
  {'query': 'Do you have any dizziness, nausea, or vomiting?',
   'key': 'symptoms:vomiting'}],
 'unmatched': [{'query': 'How about any chest pain?',
   'key': 'symptoms:chest_pain'}]}

In [36]:
# Deterministict mapper does the same using entity extraction with SpaCy. The queries are more brief but overall accurate
det_mapper.map_history_requests(query)

{'matched': [{'query': 'dizziness', 'key': 'symptoms:dizziness'},
  {'query': 'nausea', 'key': 'symptoms:nausea'},
  {'query': 'nausea', 'key': 'symptoms:vomiting'},
  {'query': 'vomiting', 'key': 'symptoms:nausea'},
  {'query': 'vomiting', 'key': 'symptoms:vomiting'}],
 'unmatched': [{'query': 'chest pain', 'key': 'chest_pain'}]}

In [47]:
clinical_case.history.symptoms

{'dizziness': Symptom(name='Dizziness', present=True, onset='sudden, two hours prior to arrival', duration='persistent since onset', progression='constant', timing=None, severity='8/10', system='neurological', location=None, character='severe spinning sensation (vertigo)', radiation=None, alleviating_factors=[], aggravating_factors=['standing', 'moving his head'], associated_symptoms=['Nausea', 'Vomiting', 'Unsteady gait', 'Diplopia'], context='occurred while watching television', history='sudden, severe spinning sensation described as vertigo, constant and rated 8/10 in severity, associated with nausea and vomiting, worsened by standing or moving his head'),
 'unsteady_gait': Symptom(name='Unsteady gait', present=True, onset='sudden, two hours prior to arrival', duration='persistent since onset', progression=None, timing=None, severity=None, system='neurological', location=None, character=None, radiation=None, alleviating_factors=[], aggravating_factors=[], associated_symptoms=['Dizzi

In [65]:
# Other supported methods
# llm_mapper.map_physical_requests
# llm_mapper.map_imaging_requests
# llm_mapper.map_investigation_requests
# For symptoms in particular, there are also additional modifiers under [addit] that agents will need to elucidate to pinpoint the diagnosis.

query = "Tell me more about the dizziness. Does it feel like the world is spinning? And do you have other symptoms?"
requests = llm_mapper.map_history_requests(query)
requests

{'matched': [{'query': 'Tell me more about the dizziness.',
   'key': 'symptoms:dizziness'},
  {'query': 'And do you have other symptoms?',
   'key': 'symptoms:unsteady_gait'}],
 'unmatched': [{'query': 'Does it feel like the world is spinning?',
   'key': 'symptoms:vertigo'}]}

In [75]:
det_parser = DeterminsticParser(clinical_case)
llm_parser = LLMParser(clincase=clinical_case, model=model)

In [76]:
# In general, both parser versions provide the same information. However, the LLM version is more expressive for LLM / human end users
print("Deterministic version:")
print(det_parser.parse_history_requests(query, requests))
print("LLM version:")
print(llm_parser.parse_history_requests(query, requests))

Deterministic version:
The patient experiences dizziness, unsteady gait.



Negative: Vertigo.
LLM version:
The dizziness came on really suddenly and it feels very severe, but it doesn’t feel like the world is spinning. I also feel really unsteady on my feet, like I might lose my balance.


#### Examination and Metrics

The following is en example of the VivaBench evaluation framework. Each ClinicalCase is contained within an Examination, which is the test harness to test any LLM agent

In [ ]:
from vivabench import Examination
from vivabench.utils import transform_agent_trace

# We will just use the same model to test for now
examination = Examination(agent_model=model, clincase=clinical_case, examiner_model=model)
trace, stats = examination.conduct_examination()

In [18]:
# The full interaction with agent, including reasoning is preserved
for t in trace[1:]:
    if 'action' in t.lower():
        print(transform_agent_trace(t))
    else:
        print("Examiner:", t)
    print("========================================================")

Examiner: Clinical case stem: 67 year old male presenting with sudden severe dizziness, unsteadiness, and double vision.
HR 84 bpm, BP 168/94 mmHg
Please review and diagnose the patient.
Agent: The patient's sudden onset of severe dizziness, unsteadiness, and double vision suggests a possible acute neurological event, such as a stroke or transient ischemic attack, particularly involving the posterior circulation. I need to clarify the onset, associated symptoms, and any vascular risk factors.
[history] When did your symptoms start? Have you experienced any weakness, numbness, difficulty speaking, or trouble swallowing?
Examiner: My dizziness started very suddenly about two hours ago, and it’s been constant since then. I have noticed some numbness or tingling, but I haven’t had any weakness, trouble speaking, or difficulty swallowing.
Agent: The sudden onset of severe dizziness, unsteadiness, double vision, and numbness/tingling raises concern for a posterior circulation stroke or TIA. 

In [19]:
from vivabench.metrics import EvaluationMetrics
metrics = EvaluationMetrics(
    icd_embedding_path="../medical/icd_embeddings",
    icd_mapping_path="../medical/d_icd_diagnoses.csv"
)

Using preloaded embeddings for icd-10 mapping


In [76]:
# These are the metrics we calculate / track. See documentation / paper for full explanation of what they mean
metrics.load_results(gt_diagnosis=clinical_case.diagnosis, 
                     gt_differentials=clinical_case.differentials,
                     matched_keys=stats['matched_keys'],
                     unmatched_case_keys=stats['unmatched_case_keys'],
                     unmatched_request_keys=stats['unmatched_request_keys'],
                     provisional_diagnosis=stats['provisional_diagnosis'],
                     final_diagnosis=stats['final_diagnosis'],
                     full_info_diagnosis=stats['full_info_diagnosis'],
                     )
_metrics = metrics.compute_all_metrics()
print(_metrics.keys())


dict_keys(['final_top1_exact', 'final_top1_approx', 'final_top2_exact', 'final_top2_approx', 'final_top3_exact', 'final_top3_approx', 'final_top4_exact', 'final_top4_approx', 'final_top5_exact', 'final_top5_approx', 'final_confidence_score', 'provisional_top1_exact', 'provisional_top1_approx', 'provisional_top2_exact', 'provisional_top2_approx', 'provisional_top3_exact', 'provisional_top3_approx', 'provisional_top4_exact', 'provisional_top4_approx', 'provisional_top5_exact', 'provisional_top5_approx', 'provisional_confidence_score', 'full_info_top1_exact', 'full_info_top1_approx', 'full_info_top2_exact', 'full_info_top2_approx', 'full_info_top3_exact', 'full_info_top3_approx', 'full_info_top4_exact', 'full_info_top4_approx', 'full_info_top5_exact', 'full_info_top5_approx', 'full_info_confidence_score', 'hp_matched_precision', 'hp_matched_recall', 'hp_matched_f1', 'hp_matched_relevant_ordered_count', 'hp_matched_total_ordered_count', 'hp_matched_total_relevant_count', 'hp_all_precision'

#### Generating new cases
Lastly, VivaBench is meant to be expandable, in that you can throw in any clinical snippets and it would be able to generate a structured Clinical Case for you. 
However, in general we recommend including as much clinical information as possible

In [ ]:
input_vignette = """
A 58-year-old male presented to the emergency department with a 90-minute history of severe, central chest pain. The pain began suddenly while he was watching television, described as a crushing, pressure-like sensation in the lower central chest, radiating to his epigastrium and occasionally to his left shoulder. The pain was constant, rated 9/10 in severity, and associated with profuse sweating, nausea, and one episode of vomiting. He reported that the pain was not relieved by rest or sublingual nitroglycerin, which he had taken at home. He denied any preceding exertion, palpitations, or syncope.

His past medical history included hypertension, hypercholesterolemia, and stable angina for which he was on aspirin, atorvastatin, and amlodipine. He had a 40 pack-year smoking history and continued to smoke one pack per day. He consumed alcohol socially and had a sedentary lifestyle. There was a strong family history of premature coronary artery disease; his father died of a myocardial infarction at age 54.

On examination, he appeared anxious and diaphoretic. Blood pressure was 98/62 mmHg, pulse 54 bpm and regular, respiratory rate 20/min, and oxygen saturation 97% on room air. Jugular venous pressure was mildly elevated. Cardiac examination revealed a soft S4 gallop but no murmurs. Lungs were clear to auscultation. Peripheral pulses were intact, and there was no peripheral edema.

Laboratory investigations revealed elevated troponin I (2.8 ng/mL, reference <0.04 ng/mL), CK-MB 38 U/L (reference <5 U/L), and mild leukocytosis. Renal and liver function tests were within normal limits. Lipid profile showed elevated LDL cholesterol (4.2 mmol/L).

The ECG demonstrated ST-segment elevation in leads II, III, and aVF, with reciprocal ST depression in leads I and aVL, and bradycardia consistent with sinus rhythm. There was also evidence of first-degree AV block (PR interval 220 ms).

A bedside echocardiogram showed hypokinesis of the inferior wall of the left ventricle with preserved overall ejection fraction.

The diagnosis was acute inferior ST-elevation myocardial infarction (STEMI), ICD-10 code I21.1 (Acute transmural myocardial infarction of inferior wall)."""

In [79]:
from vivabench import CaseGenerator
generator = CaseGenerator(model, 
                          snomed_embedding_path="../medical/snomed_embeddings",
                          icd_embedding_path="../medical/icd_embeddings",
                          icd_mapping_path="../medical/d_icd_diagnoses.csv")

2025-05-29 22:18:22.293 | INFO     | vivabench.generator:__init__:102 - Using preloaded embeddings for SNOMED
2025-05-29 22:18:28.007 | INFO     | vivabench.generator:__init__:111 - Using preloaded embeddings for icd-10 mapping


In [80]:
gen_results = await generator.generate_case(vignette = input_vignette)

2025-05-29 22:18:58.008 | INFO     | vivabench.generator:generate_case:760 - Parsing diagnosis from vignette


In [81]:
# Remember to double check the generate results first prior to using it in evaluation!
print(ClinicalCase.model_validate_json(gen_results['output']).full_information)

58 year old male presenting with chest pain. 
==== Symptoms
### Cardiovascular
## Chest pain
- Present: True
- Onset: sudden
- Duration: 90 minutes
- Progression: constant
- Severity: 9/10
- Location: lower central chest
- Character: crushing, pressure-like
- Radiation: epigastrium and occasionally to left shoulder
- Associated symptoms: Excessive sweating, Nausea, Vomiting
- Context: occurred while watching television, not relieved by rest or sublingual nitroglycerin
- History: no preceding exertion
### Autonomic
## Excessive sweating
- Present: True
- Context: accompanied chest pain
### Gastrointestinal
## Nausea
- Present: True
- Context: accompanied chest pain
## Vomiting
- Present: True
- Onset: one episode
- Context: accompanied chest pain

==== Past Medical History
- hypertension
- hypercholesterolemia
- stable angina

Allergies: No known allergies
==== Social History
Current smoker, 40.0 pack years smoking history.
Alcohol use: Consumes alcohol socially
Exercise: Sedentary life